In [6]:
import praw
import pandas as pd

In [2]:
usrname = 'mpbot0'
cid = 'yUJmELsXRFcD6A'
csecret = 'VKquj58R_cCuv021SqH6DYkCUic'
psswd = 'abcd1234'

In [3]:
# create a Reddit instance
reddit = praw.Reddit(client_id=cid,
                     client_secret=csecret, password=psswd,
                     user_agent='PrawTut', username=usrname)

In [35]:
sublist = ['the_donald','news','politics','conservative','gaming','Showerthoughts','science','movies','wholesomememes',
           'todayilearned','theredpill','imgoingtohellforthis','cringeanarchy','shitpoliticssays',
           'atheism','wholesomegifs','aww','mademesmile','kotakuinaction','games','music','sports','funny','4chan','happy']

In [36]:
len(sublist)

25

In [37]:
full_comment = []
for sub in sublist:
    subreddit = reddit.subreddit(sub)
    
    #quarantined subreddits require an opt-in process
    try:
        next(subreddit.hot())
    except:
        print("quarantine encountered")
        subreddit.quaran.opt_in()
        
    subreddit = reddit.subreddit(sub)
    top = subreddit.top('week',limit=20)
    print("Current Subreddit: " + sub)
    count = 0
    comments = []
    for submission in top:
        if not submission.stickied:
            submission.comments.replace_more(limit=0)
            hidden_count = 0
            for comment in submission.comments.list():
                if count>=500: #required no. of comments reached
                    break
                try:
                    if comment.score_hidden == True: # score is hidden due to being new thread
                        hidden_count += 1
                        if hidden_count>10:
                            break
                        else:
                            continue
                    if comment.body in ['[removed]','[deleted]']:
                        continue
                    c = comment.body
                    score = comment.score
                    comments.append([c,score])
                except praw.exceptions.PRAWException as e:
                    pass
                count+=1  
        if count>=1000:
            break
    full_comment.append(comments)

Current Subreddit: the_donald
Current Subreddit: news
Current Subreddit: politics
Current Subreddit: conservative
Current Subreddit: gaming
Current Subreddit: Showerthoughts
Current Subreddit: science
Current Subreddit: movies
Current Subreddit: wholesomememes
Current Subreddit: todayilearned
Current Subreddit: theredpill
Current Subreddit: imgoingtohellforthis
Current Subreddit: cringeanarchy
Current Subreddit: shitpoliticssays
Current Subreddit: atheism
Current Subreddit: wholesomegifs
Current Subreddit: aww
Current Subreddit: mademesmile
Current Subreddit: kotakuinaction
Current Subreddit: games
Current Subreddit: music
Current Subreddit: sports
Current Subreddit: funny
Current Subreddit: 4chan
Current Subreddit: happy


In [39]:
print("Number of comments:")
for i in range(len(sublist)):
    print(sublist[i], len(full_comment[i]))

Number of comments:
the_donald 500
news 500
politics 500
conservative 500
gaming 500
Showerthoughts 500
science 500
movies 500
wholesomememes 500
todayilearned 500
theredpill 500
imgoingtohellforthis 500
cringeanarchy 500
shitpoliticssays 500
atheism 500
wholesomegifs 500
aww 500
mademesmile 500
kotakuinaction 500
games 500
music 500
sports 500
funny 500
4chan 500
happy 500


In [40]:
comments = []
count = 0 
for sub in full_comment:
    for comment in sub:
        comments.append([comment[0],comment[1], sublist[count]])  #body score subreddit name
    count+=1

In [41]:
df = pd.DataFrame(columns=['comment','score','subreddit'])

In [43]:
df.comment = [x[0] for x in comments]
df.score = [x[1] for x in comments]
df.subreddit = [x[2] for x in comments]

In [44]:
df.head()

,comment,score,subreddit
0,I think it would be good for him to do a natio...,491,the_donald
1,"Um I saw on my home page, r/eeeedacted that he...",137,the_donald
2,This man will never walk alone,169,the_donald
3,I am fearful that the hatred spewed nightly is...,139,the_donald
4,Amen. We love him and are all so thankful for ...,100,the_donald


In [45]:
df.to_csv("25subs_500_scored.csv", index=False)